In [1]:
import syft as sy
import os
from syft import ActionObject
from collections import defaultdict

Start this using

`docker compose --profile blob-storage --file docker-compose.multinode.yml --file docker-compose.dev.yml up`

In [2]:
client = sy.login(url="http://localhost:80", email="info@openmined.org", password="changethis")

Logged into <high-side: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# Mount storage container with Helm azure container

In [3]:
client.api.services.blob_storage.mount_azure(
    account_name='helmprojectstorage',
    container_name='helm',
    account_key=os.environ["HELM_STORAGE_ACCOUNT_KEY"],
    bucket_name='helmazurebucket',
)

SyftSuccess: Mounting Azure Successful!

In [4]:
blob_files = client.api.services.blob_storage.get_files_from_bucket(bucket_name='helmazurebucket')

In [5]:
blob_files

[syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile,
 syft.types.blob_storage.BlobFile]

# Start workers

In [6]:
client.worker.start_workers(n=3)

SyftSuccess: 3 workers added

In [7]:
client.worker.list()

[syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker]

# Create Dataset

In [8]:
train_file = [f for f in blob_files if "train-00" in f.file_name][0]
scenario_file = [f for f in blob_files if "scenario_data" in f.file_name][0]

In [9]:
helm_dataset = sy.Dataset(
    name="Helm Dataset",
    asset_list=[
        sy.Asset(
            name="helm train data",
            data=ActionObject.from_obj([train_file]),
            mock=sy.ActionObject.empty()
        ),
        sy.Asset(
            name="helm test data",
            data=ActionObject.from_obj([scenario_file]),
            mock=sy.ActionObject.empty()
        )
    ]
)

In [10]:
client.upload_dataset(helm_dataset)

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]: y


  0%|                                                                                                                                  | 0/2 [00:00<?, ?it/s]

Uploading: helm train data


 50%|█████████████████████████████████████████████████████████████                                                             | 1/2 [00:00<00:00,  2.15it/s]

Uploading: helm test data


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.79it/s]


SyftSuccess: Dataset uploaded to 'high-side'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [11]:
helm_ds = client.datasets["Helm Dataset"]
helm_train_files = helm_ds.assets["helm train data"]
helm_test_files = helm_ds.assets["helm test data"]

SyftWarning: You're performing an operation on high side domain, which could host datasets with private information.

# Syft functions

In [28]:
x=4

In [30]:
f"{x+0.12432:.2f}"

'4.12'

In [12]:
@sy.syft_function()
def compute_document_data_overlap(scenario_file, input_files, n):
    print("starting overlap computation")

    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json
    import time

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for i, scenario in enumerate(light_scenarios):
            if i%20 == 0:
                print(f"n_gram indexing progress: {(i/len(light_scenarios))*100:.2f}%")
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # SETUP
    print("preparing scenarios and creating indexes")
    start = time.time()
    light_scenarios = []
    for i, (bytes_read, light_scenario_json) in enumerate(scenario_file.iter_lines(progress=True)):
        if i % 20 == 0:
            print(f"scenario creation progress: {(bytes_read/scenario_file.file_size)*100:.2f}%")

        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        scenario_spec = str(light_scenario_key_dict["scenario_spec"])

        light_scenario_key = scenario_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
    print(f"Finished creating scenarios ({time.time()-start}s)")
    
    print("Creating indexes")
    
    start = time.time()
    stats_key_counts = defaultdict(int)
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    print(f"Finished creating indexes ({time.time()-start}s)")
        
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    start = time.time()
    
    for input_file in input_files:
        for i, (bytes_read, line) in enumerate(input_file.iter_lines(progress=True)):
            if i%1000 == 0:
                print(f"computing overlap progress: {(bytes_read / input_file.file_size) * 100:.2f}%")            
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
    print(f"Finished computing overlap ({time.time()-start}s)")
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

SyftSuccess: Syft function 'compute_document_data_overlap' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [13]:
client.code.submit(compute_document_data_overlap)

SyftSuccess: User Code Submitted

In [14]:
@sy.syft_function_single_use(input_files=helm_train_files, scenario_files=helm_test_files)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


SyftSuccess: Syft function 'main_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [15]:
client.code.request_code_execution(main_function)

syft.service.request.request.Request

In [16]:
client.requests[-1].approve()

SyftWarning: You're approving a request on high side domain which may host datasets with private information.

Would you like to proceed? [y/n]: y
Request approved for domain high-side


SyftSuccess: Request bf76e4938fa74f7fa826b33fe955916d changes applied

In [17]:
job = client.code.main_function(input_files=helm_train_files,
                                scenario_files=helm_test_files,
                                blocking=False)

# Inspect Jobs and get results

In [19]:
job

```python
class Job:
    id: UID = c3722f48c0eb487c81425194faf3d372
    status: completed
    has_parent: False
    result: ActionDataEmpty UID: 332e1575cc0f437ba251d873563dc88e <None>
    logs:

0 
JOB COMPLETED
    
```

In [20]:
# job.subjobs

In [27]:
job.subjobs[0]

```python
class Job:
    id: UID = b1b8d8f0fac7415c9f3e1f54cafcac61
    status: processing
    has_parent: True
    result: None
    logs:

0 starting overlap computation
1 preparing scenarios and creating indexes
2 scenario creation progress: 0.0001514909998688016%
3 scenario creation progress: 1.51293402912451%
4 scenario creation progress: 8.62051451241565%
5 scenario creation progress: 16.406337566700422%
6 scenario creation progress: 20.851811018917626%
    
```

In [48]:
# job.wait().get()

In [83]:
job.subjobs

[syft.service.job.job_stash.Job]

In [81]:
job.subjobs[0].logs()

starting overlap computation
preparing scenarios and creating indexes
Finished creating indexes (1853.709231376648s)



In [44]:
results = [j.wait().get() for j in job.subjobs]

In [45]:
#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

[(defaultdict(set,
              {'philosophy_test_5': {'id104',
                'id106',
                'id112',
                'id115',
                'id116',
                'id118',
                'id122',
                'id127',
                'id129',
                'id136',
                'id138',
                'id146',
                'id148',
                'id153',
                'id156',
                'id161',
                'id165',
                'id170',
                'id173',
                'id174',
                'id177',
                'id186',
                'id189',
                'id194',
                'id200',
                'id213',
                'id216',
                'id218',
                'id220',
                'id224',
                'id226',
                'id230',
                'id239',
                'id248',
                'id256',
                'id257',
                'id261',
                'id263',
                'id267',
                'id270',
                'id273',
                'id274',
                'id279',
                'id280',
                'id285',
                'id287',
                'id289',
                'id291',
                'id293',
                'id296',
                'id299',
                'id300',
                'id302',
                'id306',
                'id310',
                'id311',
                'id319',
                'id328',
                'id329',
                'id332',
                'id334',
                'id335',
                'id346',
                'id349',
                'id50',
                'id51',
                'id53',
                'id56',
                'id58',
                'id62',
                'id65',
                'id66',
                'id67',
                'id78',
                'id81',
                'id82',
                'id83',
                'id84',
                'id86',
                'id92',
                'id93',
                'id94',
                'id95',
                'id96'},
               'anatomy_test_5': {'id105',
                'id108',
                'id109',
                'id110',
                'id111',
                'id115',
                'id116',
                'id125',
                'id131',
                'id134',
                'id136',
                'id138',
                'id139',
                'id140',
                'id144',
                'id145',
                'id150',
                'id151',
                'id21',
                'id24',
                'id26',
                'id30',
                'id31',
                'id33',
                'id34',
                'id41',
                'id45',
                'id47',
                'id50',
                'id52',
                'id53',
                'id55',
                'id58',
                'id59',
                'id60',
                'id61',
                'id68',
                'id72',
                'id76',
                'id77',
                'id79',
                'id82',
                'id83',
                'id84',
                'id85',
                'id87',
                'id88',
                'id90',
                'id91',
                'id92',
                'id93',
                'id94',
                'id95',
                'id96',
                'id99'},
               'philosophy_valid_5': {'id12',
                'id16',
                'id17',
                'id18',
                'id28',
                'id32',
                'id34',
                'id36',
                'id37',
                'id5',
                'id6',
                'id7'},
               'philosophy_train_5': {'id1', 'id3', 'id4'},
               'anatomy_train_5': {'id1', 'id4'},
               'anatomy_valid_5': {'id15', 'id5', 'id7', 'id9'}}),
  defaultdict(set,
              {'ph

In [46]:
results[0]

(defaultdict(set,
             {'philosophy_test_5': {'id104',
               'id106',
               'id112',
               'id115',
               'id116',
               'id118',
               'id122',
               'id127',
               'id129',
               'id136',
               'id138',
               'id146',
               'id148',
               'id153',
               'id156',
               'id161',
               'id165',
               'id170',
               'id173',
               'id174',
               'id177',
               'id186',
               'id189',
               'id194',
               'id200',
               'id213',
               'id216',
               'id218',
               'id220',
               'id224',
               'id226',
               'id230',
               'id239',
               'id248',
               'id256',
               'id257',
               'id261',
               'id263',
               'id267',
               'id270',
 

# Aggregate

In [35]:
stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.split('_')
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'scenario_spec': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)


In [36]:
from pprint import pprint
pprint(all_data_overlap_stats)

[{'data_overlap_stats_key': {'light_scenario_key': {'split': 'train',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': [],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 10},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'valid',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id12'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 68},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'test',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id328'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 